In [ ]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)
%load_ext autoreload
%autoreload 2

# Deepagent for Research

We will use the `deepagents` package to create a research agent. When using the `deepagents` package, it's important to: 

1. Understand the native tools available
2. Supply task-specific tools
3. Supply task-specific instructions
4. Supply task-specific sub-agents

## Task-Specific Tools 

You can see an overview of the native tools in the [deepagents package README](https://github.com/langchain-ai/deepagents?tab=readme-ov-file#model) as well as the [quickstarts README](https://github.com/langchain-ai/deepagents-quickstarts). We'll extend this with two task-specific tools. 

### Search Tool 

There are different search tools that we can use. For example, we can use [Tavily](https://www.tavily.com/) to search for relevant URLs, then fetches the full webpage content.

### Think Tool 

We'll supply a [think tool](https://www.anthropic.com/engineering/claude-think-tool), which is a useful way to help audit agent decision making. 

In [ ]:
from research_agent.tools import tavily_search, think_tool, contextual_search
tools = [tavily_search, think_tool, contextual_search]

In [ ]:
# Test contextual_search tool
print("Testing contextual_search tool...")
print("=" * 80)

# Test with a sample query (adjust based on your knowledge base)
test_query = "what is microsofts revenue?"
print(f"\nQuery: {test_query}\n")

try:
    result = contextual_search.invoke({"query": test_query, "max_results": 5})
    print(result)
    print("\n" + "=" * 80)
    print("✅ contextual_search tool is working!")
except Exception as e:
    print(f"\n❌ Error testing contextual_search: {e}")
    print("\nMake sure you have:")
    print("1. CONTEXTUAL_AI_API_KEY set in your .env file")
    print("2. CONTEXTUAL_AI_AGENT_ID set in your .env file")
    print("3. The 'contextual' package installed: pip install contextual")


In [ ]:
# Optional: Inspect raw ContextualAI response structure
# This helps you understand what data is available
import os
from contextual import ContextualAI

api_key = os.getenv("CONTEXTUAL_AI_API_KEY")
agent_id = os.getenv("CONTEXTUAL_AI_AGENT_ID")

if api_key and agent_id:
    client = ContextualAI(api_key=api_key)
    
    # Get raw response WITH content text included
    query_result = client.agents.query.create(
        agent_id=agent_id,
        messages=[{"content": "what is microsofts revenue?", "role": "user"}],
        retrievals_only=True,
        include_retrieval_content_text=True  # This is the key parameter!
    )
    
    print("Raw QueryResponse structure:")
    print(f"Conversation ID: {query_result.conversation_id}")
    print(f"Number of retrieval contents: {len(query_result.retrieval_contents)}")
    print("\n" + "=" * 80)
    
    # Show first result details
    if query_result.retrieval_contents:
        first_result = query_result.retrieval_contents[0]
        print("\nFirst result structure:")
        print(f"  - doc_name: {getattr(first_result, 'doc_name', 'N/A')}")
        print(f"  - content_id: {getattr(first_result, 'content_id', 'N/A')}")
        print(f"  - doc_id: {getattr(first_result, 'doc_id', 'N/A')}")
        print(f"  - page: {getattr(first_result, 'page', 'N/A')}")
        print(f"  - format: {getattr(first_result, 'format', 'N/A')}")
        print(f"  - score: {getattr(first_result, 'score', 'N/A')}")
        print(f"  - content_text available: {getattr(first_result, 'content_text', None) is not None}")
        if hasattr(first_result, 'content_text') and first_result.content_text:
            print(f"\n  - content_text preview (first 300 chars):")
            print(f"    {first_result.content_text[:300]}...")
        else:
            print(f"\n  ⚠️  content_text is None or empty")
            print(f"  Make sure you're using include_retrieval_content_text=True in the query!")
else:
    print("⚠️  ContextualAI credentials not found. Skipping raw response inspection.")


## Task-Specific Instructions
 
Next, let's define task specific instructions using [a few prompting techniques for agents](https://youtu.be/XSZP9GhhuAc?si=zowpViL-2j-vI9hA):

### 1. Think Like The Agent
What instructions would you give a new work colleague?
- **Read the question carefully** - What specific information does the user need?
- **Start with broader searches** - Use broad, comprehensive queries first
- **After each search, pause and assess** - Do I have enough to answer? What's still missing?
- **Execute narrower searches as you gather information** - Fill in the gaps.

### 2. Concrete Heuristics (Prevent "Spin-Out" on excessive tool calls)
Use **Hard Limits** to prevent the research agent from calling tools excessively:
- **Stop when you can answer confidently** - Don't keep searching for perfection.
- **Give it budgets** - Use 2-3 search tool calls for simple queries. Use up to 5 for complex queries.
- **Limit** - Always stop after 5 search tool calls if you cannot find the right source(s).

### 3. Show your thinking
After each search tool calling, use [`think_tool` to analyze the results](https://www.anthropic.com/engineering/claude-think-tool):
- What key information did I find? 
- What's missing?
- Do I have enough to answer the question comprehensively?
- Should I search more or provide my answer?

In [ ]:
from datetime import datetime
from utils import show_prompt, format_messages
from research_agent.prompts import (
    RESEARCHER_INSTRUCTIONS,
    RESEARCH_WORKFLOW_INSTRUCTIONS,
    SUBAGENT_DELEGATION_INSTRUCTIONS,
)

In [ ]:
show_prompt(RESEARCHER_INSTRUCTIONS)

## Task-Specific Sub-Agents

You can specify [custom subagents](https://github.com/langchain-ai/deepagents?tab=readme-ov-file#subagents) as a means of context isolation. 

Here's well define a sub-agent that can search the web for information. 

In [ ]:
# Get current date
current_date = datetime.now().strftime("%Y-%m-%d")

# Create research sub-agent
research_sub_agent = {
    "name": "research-agent",
    "description": "Delegate research to the sub-agent researcher. Only give this researcher one topic at a time.",
    "system_prompt": RESEARCHER_INSTRUCTIONS.format(date=current_date),
    "tools": [contextual_search, tavily_search, think_tool],
}

## Putting it all together

### Instructions

Now, we can look at all of our instructions together. 

In [ ]:
# Limits
max_concurrent_research_units = 5
max_researcher_iterations = 2

# Combine orchestrator instructions (RESEARCHER_INSTRUCTIONS only for sub-agents)
INSTRUCTIONS = (
    RESEARCH_WORKFLOW_INSTRUCTIONS
    + "\n\n"
    + "=" * 80
    + "\n\n"
    +  SUBAGENT_DELEGATION_INSTRUCTIONS.format(
        max_concurrent_research_units=max_concurrent_research_units,
        max_researcher_iterations=max_researcher_iterations,
    )
)

show_prompt(INSTRUCTIONS)

### Create the agent

Now, we create our deepagent with these components. 

In [ ]:
from IPython.display import Image, display
from deepagents import create_deep_agent
from langchain.chat_models import init_chat_model
#from langchain_google_genai import ChatGoogleGenerativeAI

# Model Gemini 3 
#model = ChatGoogleGenerativeAI(model="gemini-3-pro-preview")

# Model Claude 4.5
model = init_chat_model(model="anthropic:claude-sonnet-4-5-20250929", temperature=0.0)

In [ ]:
# Create the agent
agent = create_deep_agent(
      model=model,
      tools=tools, 
      system_prompt=INSTRUCTIONS,
      subagents=[research_sub_agent],
  )
  
# Show the agent
display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
simple_query = "Based on NVIDIA's past performance, what is their best strategy for the future?"
complex_query = '''
I need a reality check on NVIDIA's three biggest bets: Automotive, Data Center, and Sovereign AI.
For each of these three sectors, can you compare what NVIDIA claims in their official documents (their stated strategy and risks) against what is actually happening in the market right now?
I'm specifically looking for a 'Fact vs. Reality' analysis: does recent news and competitor activity (like from AMD or Huawei) validate their internal confidence, or are there cracks in the narrative?
'''

In [ ]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": complex_query,
            }
        ],
    }, 
)
format_messages(result["messages"])

from utils import extract_todos_from_state, format_todos_as_markdown

todos = extract_todos_from_state(result)
research_question = result["messages"][0].content
markdown = format_todos_as_markdown(todos, research_question)
print(markdown)

# Or read the automatically saved research plan
from deepagents.backends.utils import file_data_to_string
if '/research_plan.md' in result.get("files", {}):
    plan = file_data_to_string(result["files"]['/research_plan.md'])
    show_prompt(plan, title="Research Plan")

In [ ]:
from deepagents.backends.utils import file_data_to_string

# Convert a specific file to string
file_content = file_data_to_string(result["files"]['/final_report.md'])
show_prompt(file_content) 

In [ ]:
from deepagents.backends.utils import file_data_to_string

# Read from agent's in-memory filesystem
file_content = file_data_to_string(result["files"]['/final_report.md'])
show_prompt(file_content)

# Save to disk (add this)
with open('final_report.md', 'w') as f:
    f.write(file_content)
print("✅ Saved to: final_report.md")

# Also save research plan if it exists
if '/research_plan.md' in result.get("files", {}):
    plan_content = file_data_to_string(result["files"]['/research_plan.md'])
    with open('research_plan.md', 'w') as f:
        f.write(plan_content)
    print("✅ Saved to: research_plan.md")

In [ ]:
# Count and display sub-agent usage
from utils import print_subagent_summary

# Print a nice summary
print_subagent_summary(result)

Trace: 

Simple query: https://smith.langchain.com/public/61058724-5b42-4cb2-8d03-0a071a10876d/r


Complex query: https://smith.langchain.com/public/160e15b3-0e18-451b-898e-463d7f1177ef/r